In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.optim import SGD
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

seed = 3047
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [2]:
!gdown 1o0m3jyfmetUOJ146TqHuEGUWwQyC7JXV
!gdown 1B5OC3R0yM8F7yjoYOKu3t08QZalcr7DC
!gdown 1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2

Downloading...
From: https://drive.google.com/uc?id=1o0m3jyfmetUOJ146TqHuEGUWwQyC7JXV
To: /content/train.csv
100% 6.54M/6.54M [00:00<00:00, 38.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B5OC3R0yM8F7yjoYOKu3t08QZalcr7DC
To: /content/val.csv
100% 665k/665k [00:00<00:00, 150MB/s]
Downloading...
From: https://drive.google.com/uc?id=1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2
To: /content/X_test
100% 3.57M/3.57M [00:00<00:00, 260MB/s]


In [83]:
class SVM(nn.Module):
  def __init__(self):
    # TODO design your model
    super(SVM, self).__init__() 
    self.w = nn.Parameter(torch.randn((1, 8192)).to(torch.float32))
    self.f = nn.Sequential(
                  nn.Linear(107, 8192),
                  nn.Dropout(0.5),
                )
  def transform(self, x):
    x = self.f(x)
    return x
  def kernel(self, x):
    pass
  def forward(self, x):
    f = torch.matmul(self.transform(x), self.w.T)
    
    return f

In [51]:
class HingeLoss(nn.Module):
  def __init__(self, C):
    super(HingeLoss, self).__init__()  
    self.C = C
  def forward(self, y, f):
    loss = 0
    for i in range(y.shape[0]):
      loss = loss + C * max(0, 1-y[i]*f[i])
    return loss


In [14]:
class TrainDataset(Dataset):
  def __init__(self, split, mu=None, std=None):
    X = pd.read_csv(f"{split}.csv")
    
    Y = X['y'].values.reshape(-1) * 2 - 1
    X['y']
    X = self.normalize(X.drop(['y'], axis=1), mu, std)
    X = np.concatenate((X, np.ones((X.shape[0], 1))), 1)
    self.Y = torch.from_numpy(Y).to(torch.float32)
    self.X = torch.from_numpy(X).to(torch.float32)

  def normalize(self, X, mu=None, std=None):
    for i in ([0,1,3,4,5]):
      X.iloc[:,i] = (X.iloc[:,i] - X.iloc[:,i].mean()) / X.iloc[:,i].std()
    
    return X
  
  def __len__(self):
    return self.X.size(0)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

class TestDataset(Dataset):
  def __init__(self, mu=None, std=None):
    X = pd.read_csv("X_test")
    X = self.normalize(X, mu, std)
    X = np.concatenate((X, np.ones((X.shape[0], 1))), 1)
    self.X = torch.from_numpy(X).to(torch.float32)

  def normalize(self, X, mu_x, std_x):
    for i in ([0,1,3,4,5]):
      X.iloc[:,i] = (X.iloc[:,i] - X.iloc[:,i].mean()) / X.iloc[:,i].std()
    
    return X
  
  def __len__(self):
    return self.X.size(0)

  def __getitem__(self, idx):
    return self.X[idx]

In [62]:
def train(train_data, val_data, model, optim, C, device='cuda:0'):
    epoch = 15
    objective = HingeLoss(C)
    steps = 0
    best = 0
    print(epoch * len(train_data))
    for e in range(epoch):
      for tr in train_data:
        steps += 1
        x_train, y_train = tr
        x_train, y_train = x_train.to(device), y_train.to(device)
        pred = model(x_train).squeeze(1)
        loss = objective(pred, y_train) + 1 / 2 * torch.sum(model.w[:-1] ** 2)
        
        optim.zero_grad()
        loss.backward()
        optim.step()

        
        if steps % 100 == 0:
          model.eval()
          with torch.no_grad():
            acc = []
            for val in val_data:
              x_val, y_val = val
              x_val , y_val = x_val.to(device), y_val.to(device)
              pred = model(x_val).squeeze(1)
              pred = (pred > 0) * 2 - 1
              
              result = (y_val == pred)
              acc += [(float(result.sum()) / result.size(0))]
            acc = sum(acc) / len(acc)
            print(f'Steps {steps}| Train Loss = {loss.item()}| Val acc = {acc}')
            if acc > best:
              torch.save(model.state_dict(), 'best.ckpt')
              best = acc        
          model.train()
    return model

In [79]:
lr = 0.0001
batch = 32
C = 1
device = 'cuda:0'

In [84]:
trainset = TrainDataset('train')
devset = TrainDataset('val')
testset = TestDataset()

train_dataloader = DataLoader(trainset, batch, True, drop_last=False)
val_dataloader = DataLoader(devset, 1, False)
test_dataloader = DataLoader(testset, 1, False)

model = SVM().to(device) 
optim = SGD(model.parameters(), lr)
model = train(train_dataloader, val_dataloader, model, optim, C, device)

13860
Steps 100| Train Loss = 159.05892944335938| Val acc = 0.842
Steps 200| Train Loss = 122.18543243408203| Val acc = 0.8256666666666667
Steps 300| Train Loss = 177.4066925048828| Val acc = 0.8376666666666667
Steps 400| Train Loss = 189.41549682617188| Val acc = 0.8383333333333334
Steps 500| Train Loss = 164.84390258789062| Val acc = 0.8266666666666667
Steps 600| Train Loss = 268.3534851074219| Val acc = 0.794
Steps 700| Train Loss = 117.44501495361328| Val acc = 0.8226666666666667
Steps 800| Train Loss = 71.41488647460938| Val acc = 0.836
Steps 900| Train Loss = 40.548973083496094| Val acc = 0.7996666666666666
Steps 1000| Train Loss = 66.96087646484375| Val acc = 0.8336666666666667
Steps 1100| Train Loss = 73.95323944091797| Val acc = 0.8343333333333334
Steps 1200| Train Loss = 298.15496826171875| Val acc = 0.8156666666666667
Steps 1300| Train Loss = 103.84503173828125| Val acc = 0.8036666666666666
Steps 1400| Train Loss = 345.66259765625| Val acc = 0.7643333333333333
Steps 1500| Tr

In [85]:
best_model = model
best_model.load_state_dict(torch.load('best.ckpt'))
best_model = best_model.eval()

y_test = []
for x in test_dataloader:
  x = x.to(device)
  y = best_model(x)
  y_test.append(((y > 0) * 1).item())



In [86]:
import csv
with open('predict.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['id','label'])
    for i in range(len(y_test)):
      writer.writerow( [i + 1, int(y_test[i])] )